In [1]:
import zipfile
from zipfile import ZipFile
import pandas as pd
import numpy as np
import copy
import re
import os
import netCDF4 as nc
from matplotlib import pyplot as plt
import xarray as xr
import os
from matplotlib.patches import Rectangle
import warnings
warnings.filterwarnings("ignore")
import matplotlib.pyplot as plt
from mpl_toolkits.basemap import Basemap
from matplotlib.patches import Polygon
# import rioxarray as rxr
from osgeo import gdal
import geopandas as gpd
from rasterio import Affine
from rasterstats import zonal_stats

In [2]:
pathpc = r'D:\Mosq Proj Data\Vegetation Dataset'
# Dataset was taken from Copernicus Climate Data store, however it is too much data to upload. Link below and few files included
# https://cds.climate.copernicus.eu/cdsapp#!/dataset/satellite-lai-fapar?tab=overview
ds = xr.open_dataset(r"D:\Mosq Proj Data\Vegetation Dataset\c3s_LAI_20020110000000_GLOBE_VGT_V3.0.1.nc")
data = ds['LAI'].data

In [3]:
gdf = gpd.read_file(r"C:\Users\gokul\Downloads\QGIS Natural Earth\Toronto SHP\GTA.shp") # from the GTA zip file
merged = gdf.geometry.unary_union
gdf_merged = gpd.GeoDataFrame(geometry=[merged])
gdf_merged = gdf_merged.set_crs('EPSG:4326')
gdf_m = gdf_merged.to_crs('EPSG:3857')
gdf_m['area_m2'] = gdf_m.geometry.area
areaofshp = gdf_m['area_m2']

In [4]:
# Set paths for vegetation dataset and Toronto shapefile
path = r'D:\Mosq Proj Data\Vegetation Dataset'
basedir = path
path_poly = r"C:\Users\gokul\Downloads\QGIS Natural Earth\Toronto SHP\GTA.shp"
average = {}
filepathname = []
count = []

gdf = gdf_merged
geom = gdf.geometry.values[0]

# List all files in the vegetation dataset directory
for file in os.listdir(basedir):
    filepathname.append(os.path.join(basedir, file))
for path_nc in filepathname:

    ds = xr.open_dataset(path_nc)
    ds = ds.assign_coords(lon=(((ds.lon + 180) % 360) - 180)).sortby('lon')

    # Open file with GDAL and adjust geotransformation
    gdobj = gdal.Open(path_nc)
    gt = list(gdobj.GetGeoTransform()) 
    gt[1] = np.mean(np.diff(ds['lon']))
    gt[0] = -180 - (gt[1]/2)
    gt[5] = np.mean(np.diff(ds['lat']))
    gt[3] = 80 - (gt[5]/2)
    affine = Affine.from_gdal(*gt)
    
    # Set variable of interest
    var = 'LAI'

    # Process each time step in the dataset
    for t in ds.time.data:
        array = np.squeeze(ds[var].values)
        # Perform zonal statistics
        zstats = zonal_stats(geom, array, affine=affine, stats="count", nodata=np.nan, all_touched=True, raster_out=True)
        mini = zstats[0]['mini_raster_array']

        # Handle scenarios based on pixel count
        if zstats[0]['count'] < 20:
            f_overlap = np.zeros(mini.shape)
            for i in range(mini.shape[0]):
                for j in range(mini.shape[1]):
                    if mini.mask[i,j]:
                        continue
                    # Calculate pixel geometry and overlay with polygon
                    ul = (ma[2] + (j * ma[0]), ma[5] + (i * ma[4]))
                    pixel = Polygon([ul, (ul[0] + ma[0], ul[1]), (ur[0], ur[1] + ma[4]), (lr[0] - ma[0], lr[1]), ul])
                    f_overlap[i,j] = geom.intersection(pixel).area/pixel.area
            # Calculate weighted average
            mini = np.where(mini < 0, 0, mini)
            mini = np.where(mini == 0, np.nan, mini)
            average[t] = np.ma.average(mini, weights=f_overlap)
            count.append(np.count_nonzero(~np.isnan(mini)))
        else:
            # Calculate average for sufficient pixel count
            mini = np.where(mini < 0, 0, mini)
            mini = np.where(mini == 0, np.nan, mini)
            count.append(np.count_nonzero(~np.isnan(mini)))
            average[t] = np.nanmean(mini)

# insert the results into a dataframe and remove nan
Vegetation_Data_Compiled=pd.DataFrame()
Data = pd.Series(list((average.values())))
Date = pd.Series(list((average.keys())))
Vegetation_Data_Compiled['LAI']=Data
Vegetation_Data_Compiled['Date']=Date
Vegetation_Data_Compiled['Count']=count

In [5]:
Vegetation_Data_Compiled_Copy=copy.deepcopy(Vegetation_Data_Compiled)
Vegetation_Data_Compiled_Copy=Vegetation_Data_Compiled_Copy.reset_index()
# Average by month as this dataset is the monthly average applied daily
Vegetation_Data_Compiled_Copy=Vegetation_Data_Compiled_Copy.groupby(pd.Grouper(key='Date', freq='M')).mean() 
Vegetation_Data_Compiled_Copy=Vegetation_Data_Compiled_Copy.reset_index()

In [6]:
Monthly_dates = pd.date_range(start='2002-01-01', end='2018-12-31', freq='MS')
Vegetation_Data_Compiled_Copy.set_index(Monthly_dates, inplace=True)

# resample to daily frequency and forward fill missing values
Daily_dates = pd.date_range(start='2002-01-01', end='2019-01-01', freq='D')
df_daily = Vegetation_Data_Compiled_Copy.resample('D').ffill().reindex(Daily_dates)

df_daily.loc[df_daily['LAI'].isna(), 'LAI'] = df_daily.loc['2018-12-01', 'LAI']
df_daily.loc[df_daily['Count'].isna(), 'Count'] = df_daily.loc['2018-12-01', 'Count']

In [7]:
df_daily2=copy.deepcopy(df_daily)

In [8]:
df_daily2.reset_index(inplace=True)
df_daily2['Date'] = df_daily2['index']
df_daily2.drop('index', axis=1, inplace=True)
df_daily2['Date']=df_daily2['level_0']
df_daily2=df_daily2.drop(columns=['level_0'])

In [9]:
df_daily3=copy.deepcopy(df_daily2)
df_daily3.drop(df_daily3.index[-1], inplace=True)

In [10]:
# Getting some statistics for the data we extracted, see if the counts and whatnot are good to proceed or if we need to rerun.
df_daily3['Date'] = pd.to_datetime(df_daily3['Date'])

# Monthly breakdown - mean, min, max and the sum for counts
df_daily3['Month'] = df_daily3['Date'].dt.month
veg_count_stats_by_month = df_daily3.groupby('Month')['Count'].agg(['mean', 'min', 'max'])

# Seeing the count breakdown for each month. Do some months have more count than other months?
veg_total_counts_by_month = df_daily3.groupby('Month')['Count'].sum()
veg_percent_counts_by_month = (veg_total_counts_by_month / veg_total_counts_by_month.sum()) * 100

# How many counts are below average vs minimum
below_average_by_month = df_daily3.groupby('Month')['Count'].apply(lambda x: (x < x.mean()).sum() / len(x) * 100)
at_minimum_by_month = df_daily3.groupby('Month')['Count'].apply(lambda x: (x == x.min()).sum() / len(x) * 100)

# Getting ready to relabel and show figures/data
veg_count_stats_by_month = veg_count_stats_by_month.merge(veg_percent_counts_by_month, on='Month')
veg_count_stats_by_month = veg_count_stats_by_month.merge(below_average_by_month, on='Month')
veg_count_stats_by_month = veg_count_stats_by_month.merge(at_minimum_by_month, on='Month')
veg_count_stats_by_month.rename(columns={'mean': 'Mean Count', 'min': 'Min Count', 'max': 'Max Count', 'Count_x': 'Percentage of Total Counts', 'Count_y': 'Percentage of Data Below Average Count', 'Count': 'Percentage of Data at Minimum Count'}, inplace=True)
veg_count_stats_by_month = veg_count_stats_by_month.drop('Percentage of Total Counts', axis=1)
total_data_by_month = df_daily3.groupby('Month')['Count'].count()
percent_data_by_month = (total_data_by_month / total_data_by_month.sum()) * 100

veg_count_stats_by_month = veg_count_stats_by_month.merge(percent_data_by_month, on='Month')
veg_count_stats_by_month.rename(columns={'Count': 'Percentage of Total Data'}, inplace=True)

veg_count_stats_by_month = veg_count_stats_by_month.reset_index()

print('\nCount Statistics by Month Runoff:')
print(veg_count_stats_by_month)


Count Statistics by Month Runoff:
    Month    Mean Count     Min Count  Max Count  \
0       1   9919.235294    832.000000    15494.0   
1       2   6359.703125    504.000000    10791.0   
2       3   9110.333333    496.000000    13867.0   
3       4  16213.470588   6735.000000    19491.0   
4       5  18458.441176  18188.666667    19039.0   
5       6  18621.078431  18364.000000    19709.0   
6       7  18693.196078  18266.000000    20251.0   
7       8  18580.431373  18364.666667    19495.0   
8       9  18545.117647  17698.000000    19329.0   
9      10  18675.333333  18195.000000    19885.0   
10     11  18454.156863  17417.000000    20281.0   
11     12  12378.098039    296.000000    17939.0   

    Percentage of Data Below Average Count  \
0                                41.176471   
1                                35.416667   
2                                47.058824   
3                                29.411765   
4                                76.470588   
5           

In [12]:
df_daily3[df_daily3['Month']<2]


,Date,LAI,Count,Month
0,2002-01-01,0.291881,15420.0,1
1,2002-01-02,0.291881,15420.0,1
2,2002-01-03,0.291881,15420.0,1
3,2002-01-04,0.291881,15420.0,1
4,2002-01-05,0.291881,15420.0,1
...,...,...,...,...
5870,2018-01-27,0.053392,1514.0,1
5871,2018-01-28,0.053392,1514.0,1
5872,2018-01-29,0.053392,1514.0,1
5873,2018-01-30,0.053392,1514.0,1


In [13]:
df_daily2

,Date,LAI,Count
0,2002-01-01,0.291881,15420.0
1,2002-01-02,0.291881,15420.0
2,2002-01-03,0.291881,15420.0
3,2002-01-04,0.291881,15420.0
4,2002-01-05,0.291881,15420.0
...,...,...,...
6205,2018-12-28,0.038611,557.0
6206,2018-12-29,0.038611,557.0
6207,2018-12-30,0.038611,557.0
6208,2018-12-31,0.038611,557.0


In [14]:
df_daily2.to_excel(r'Path\To\File\VegetationDataPolygon.xlsx') 